# Reinforcement Learning control strategies for Electric Vehicles fleet Virtual Power Plants
Thesis based on the development of a RL agent that manages a VPP through EVs charging stations in an household environment. Main optimization objectives of the VPP are: Valley filling, peak shaving and zero resulting load over time. Main action performed to reach objectives are: storage of Renewable energy resources and power push in the grid at high demand times. The development of the Virtual Power Plant environment is based on the ELVIS (Electric Vehicles Infrastructure Simulator) open library from DAI-Labor: https://github.com/dailab/elvis The thesis code is currently available at: (https://github.com/francescomaldonato/RL_VPP_Thesis)

Author: Francesco Maldonato

## VPP experiment tester Notebook based on EVs arrival, with StableBaselines3 trained model (Recurrent PPO) [10-15-20-25-30-35 EVs per week test]

Installing required packages and dependencies

In [1]:
%%capture
!pip install py-elvis==0.2.1
!pip install pyyaml==5.4
!pip install plotly==5.9.0
!pip install -U kaleido==0.2.1

!pip install stable-baselines3[extra]==1.6.1
!pip install stable-baselines==1.6.1
!pip install sb3-contrib==1.6.1
!pip install gym==0.20.0
!pip install -q wandb==0.13.4

In [2]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/
%ls

Cloning into 'RL_VPP_Thesis'...
remote: Enumerating objects: 443, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 443 (delta 13), reused 35 (delta 6), pack-reused 393
Receiving objects: 100% (443/443), 178.48 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (162/162), done.
Checking out files: 100% (219/219), done.
/content/RL_VPP_Thesis
Agent_trainer_notebooks/          RL_VPP_Thesis/
Algorithm_simulator_notebooks/    trained_models/
data/                             VPP_environment.py
EV_experiment_notebooks/          VPP_simulator.ipynb
Hyperparameters_sweep_notebooks/  wandb/
README.md


In [3]:
import yaml
import numpy as np
from gym import Env
from VPP_environment import VPPEnv, VPP_Scenario_config
from elvis.config import ScenarioConfig
import os
import torch
import random
import wandb
from sb3_contrib import RecurrentPPO #The available algoritmhs in sb3-contrib for the custom environment with MultiInputPolicy
from sb3_contrib.common.maskable.utils import get_action_masks
import stable_baselines3 as sb3
from stable_baselines3.common.env_checker import check_env

#Check if cuda device is available for training
print("Torch-Cuda available device:", torch.cuda.is_available())
print(sb3.get_system_info())
#!wandb --version

Torch-Cuda available device: False
OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022
Python: 3.7.14
Stable-Baselines3: 1.6.1
PyTorch: 1.12.1+cu113
GPU Enabled: False
Numpy: 1.21.6
Gym: 0.20.0

({'OS': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022', 'Python': '3.7.14', 'Stable-Baselines3': '1.6.1', 'PyTorch': '1.12.1+cu113', 'GPU Enabled': 'False', 'Numpy': '1.21.6', 'Gym': '0.20.0'}, 'OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022\nPython: 3.7.14\nStable-Baselines3: 1.6.1\nPyTorch: 1.12.1+cu113\nGPU Enabled: False\nNumpy: 1.21.6\nGym: 0.20.0\n')


In [4]:
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

## Load ELVIS YAML config file
Section where the EVs arrival simulation parameters are loaded through the Yaml config file from the 'data/config_builder/' folder.

In [5]:
#Loading paths for input data
current_folder = ''
VPP_training_data_input_path = current_folder + 'data/data_training/environment_table/' + 'Environment_data_2019.csv'
VPP_testing_data_input_path = current_folder + 'data/data_testing/environment_table/' + 'Environment_data_2020.csv'
VPP_validating_data_input_path = current_folder + 'data/data_validating/environment_table/' + 'Environment_data_2018.csv'
elvis_input_folder = current_folder + 'data/config_builder/'

#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

print(elvis_config_file)
print(VPP_config_file)

Vehicle types: <generator object ScenarioConfig.__str__.<locals>.<genexpr> at 0x7f568c26d1d0>Mean parking time: 23.99
Std deviation of parking time: 1
Mean value of the SOC distribution: 0.5
Std deviation of the SOC distribution: 0.1
Max parking time: 24
Number of charging events per week: 30
Vehicles are disconnected only depending on their parking time
Queue length: 0
Opening hours: None
Scheduling policy: Uncontrolled

{'start_date': '2022-01-01T00:00:00', 'end_date': '2023-01-01T00:00:00', 'resolution': '0:15:00', 'num_households': 4, 'solar_power': 16, 'wind_power': 12, 'EV_types': [{'battery': {'capacity': 100, 'efficiency': 1, 'max_charge_power': 150, 'min_charge_power': 0}, 'brand': 'Tesla', 'model': 'Model S', 'probability': 1}], 'charging_stations_n': 4, 'EVs_n': 30, 'EVs_n_max': 1566, 'mean_park': 23.99, 'std_deviation_park': 1, 'EVs_mean_soc': 50.0, 'EVs_std_deviation_soc': 10.0, 'EV_load_max': 44, 'EV_load_rated': 14.8, 'EV_load_min': 1, 'houseRWload_max': 10, 'av_max_ener

In [ ]:
#TESTING Environment initialization
env = VPPEnv(VPP_testing_data_input_path, elvis_config_file, VPP_config_file)
#env.plot_VPP_input_data()
#Function to check custom environment and output additional warnings if needed
check_env(env)
#env.plot_reward_functions()

In [21]:
#Loading training model, from local directory or from wandb previous trainings
RecurrentPPO_path = "trained_models/RecurrentPPO_models/model_RecurrentPPO_"

#model_id = "s37o8q0n"
model_id = "333ckz0i"
model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

# run_id_restore = "2y2dqvyn"
# model = wandb.restore(f'model_{run_id_restore}.zip', run_path=f"francesco_maldonato/RL_VPP_Thesis/{run_id_restore}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/save_util.py:169: UserWarning:

Could not deserialize object _last_lstm_states. Consider using `custom_objects` argument to replace this object.



Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Testing dataset VPP Simulation using the loaded trained model [35 EVs per week]

In [22]:
#TEST Model
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

#Save the VPP table
#VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')
VPP_table = env.VPP_table

- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  31719.34 , over-consume=kWh  47872.51 , under-consume=kWh  16153.17 , Total_cost=€  1229.53 , overcost=€  1684.28 , Av.EV_en_left=kWh  100.0 , Charging_events=  1565 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  66.72
Simulating VPP....
- VPP.Simulation results
 LOAD_INFO: Sum_Energy=KWh  -1700.17 , over-consume=KWh  1325.58 , under-consume=KWh  3025.75 , Total_cost=€  -25.99 , Overcost=€  43.4 
 EV_INFO: Av.EV_energy_leaving=kWh  61.86 , Std.EV_energy_leaving=kWh  24.23 , EV_departures =  1565 , EV_queue_left =  0
SCORE:  Cumulative_reward= 465799.55 - Step_rewars (load_t= 435018.54, EVs_energy_t= -2701.2)
 - Final_rewards (EVs_energy= 18375.07, Overconsume= -313.87, Underconsume= 317.09, Overcost= 15103.92)
Episode:1 Score:465799.54951987497
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  31064.91 , over-consume=kWh  47424.23 , under-consume=kWh  16359.32 , 

## Testing dataset VPP Simulation using the loaded trained model [30 EVs per week]

In [24]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_testing_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

#Save the VPP table
VPP_table = env.VPP_table

Charging event: 20346, Arrival time: 2022-01-01 03:00:00, Parking_time: 22.007106393057214, Leaving_time: 2022-01-02 01:00:25.583015, SOC: 0.6766515289743867, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 21910, Arrival time: 2022-12-31 16:30:00, Parking_time: 24, Leaving_time: 2023-01-01 16:30:00, SOC: 0.6128414192742176, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -21214.64 , over-consume=kWh  4947.18 , under-consume=kWh  -26161.81 , Total_cost=€  -489.75 , overcost=€  233.11
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  30167.05 , over-consume=kWh  45875.42 , under-consume=kWh  15708.37 , Total_cost=€  1158.95 , overcost=€  1621.18 , Charging_events=  1565 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  66.72
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  30742.99 , over-consume=kWh  46775.98 , under-consume=kWh  16032.99 , Total_cost=€  1

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/save_util.py:169: UserWarning:

Could not deserialize object _last_lstm_states. Consider using `custom_objects` argument to replace this object.



Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  30835.33 , over-consume=kWh  46477.22 , under-consume=kWh  15641.89 , Total_cost=€  1146.04 , overcost=€  1595.08 , Av.EV_en_left=kWh  100.0 , Charging_events=  1565 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  66.72
Simulating VPP....
- VPP.Simulation results
 LOAD_INFO: Sum_Energy=KWh  -1728.53 , over-consume=KWh  1316.0 , under-consume=KWh  3044.53 , Total_cost=€  -35.57 , Overcost=€  44.29 
 EV_INFO: Av.EV_energy_leaving=kWh  62.77 , Std.EV_energy_leaving=kWh  23.36 , EV_departures =  1560 , EV_queue_left =  1
SCORE:  Cumulative_reward= 471591.84 - Step_rewars (load_t= 430606.09, EVs_energy_t= 5920.96)
 - Final_rewards (EVs_energy= 20545.35, Overconsume= -339.39, Underconsume= 134.02, Overcost= 14724.81)
Episode:1 Score:471591.8420901467
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy

## Testing dataset VPP Simulation using the loaded trained model [25 EVs per week]

In [25]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_testing_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

#Save the VPP table
VPP_table = env.VPP_table

Charging event: 39126, Arrival time: 2022-01-01 07:30:00, Parking_time: 22.557094617966147, Leaving_time: 2022-01-02 06:03:25.540625, SOC: 0.47408835924624854, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 40429, Arrival time: 2022-12-31 20:15:00, Parking_time: 24, Leaving_time: 2023-01-01 20:15:00, SOC: 0.40579714504236836, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -21214.64 , over-consume=kWh  4947.18 , under-consume=kWh  -26161.81 , Total_cost=€  -489.75 , overcost=€  233.11
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  27025.75 , over-consume=kWh  43253.16 , under-consume=kWh  16227.41 , Total_cost=€  1053.56 , overcost=€  1521.72 , Charging_events=  1304 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  70.06
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  27340.07 , over-consume=kWh  43539.43 , under-consume=kWh  16199.37 , Total_cost=€ 

## Testing dataset VPP Simulation using the loaded trained model [20 EVs per week]

In [26]:
case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_testing_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

#Save the VPP table
VPP_table = env.VPP_table

Charging event: 54774, Arrival time: 2022-01-01 11:00:00, Parking_time: 22.63342623358456, Leaving_time: 2022-01-02 09:38:00.334441, SOC: 0.5810908333788527, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 55816, Arrival time: 2022-12-31 17:30:00, Parking_time: 22.682892629767665, Leaving_time: 2023-01-01 16:10:58.413467, SOC: 0.5995827167862343, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -21214.64 , over-consume=kWh  4947.18 , under-consume=kWh  -26161.81 , Total_cost=€  -489.75 , overcost=€  233.11
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  21614.45 , over-consume=kWh  39232.91 , under-consume=kWh  17618.46 , Total_cost=€  930.38 , overcost=€  1422.08 , Charging_events=  1043 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  75.08
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  21769.54 , over-consume=kWh  39266.4 , under-consume=kWh  17496.

## Testing dataset VPP Simulation using the loaded trained model [15 EVs per week]


In [27]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_testing_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

#Save the VPP table
VPP_table = env.VPP_table

Charging event: 67290, Arrival time: 2022-01-01 16:15:00, Parking_time: 23.498168633428214, Leaving_time: 2022-01-02 15:44:53.407080, SOC: 0.5005544474216131, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 68072, Arrival time: 2022-12-31 10:30:00, Parking_time: 23.434587977219696, Leaving_time: 2023-01-01 09:56:04.516718, SOC: 0.4039021889471574, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -21214.64 , over-consume=kWh  4947.18 , under-consume=kWh  -26161.81 , Total_cost=€  -489.75 , overcost=€  233.11
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  13732.62 , over-consume=kWh  32430.27 , under-consume=kWh  18697.65 , Total_cost=€  649.12 , overcost=€  1169.62 , Charging_events=  783 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  83.41
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  13595.15 , over-consume=kWh  32252.67 , under-consume=kWh  18657

## Testing dataset VPP Simulation using the loaded trained model [10 EVs per week]


In [28]:
#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
#case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

#print(elvis_config_file)
#print(VPP_config_file)

#TESTING Environment initialization
env = VPPEnv(VPP_testing_data_input_path, elvis_config_file, VPP_config_file)
#Function to check custom environment and output additional warnings if needed
check_env(env)

model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

#TEST Model
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

#Save the VPP table
VPP_table = env.VPP_table

Charging event: 76686, Arrival time: 2022-01-01 07:45:00, Parking_time: 24, Leaving_time: 2022-01-02 07:45:00, SOC: 0.49834440391696694, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 77468, Arrival time: 2022-12-29 17:45:00, Parking_time: 23.194821514943634, Leaving_time: 2022-12-30 16:56:41.357454, SOC: 0.43604527157406425, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -21214.64 , over-consume=kWh  4947.18 , under-consume=kWh  -26161.81 , Total_cost=€  -489.75 , overcost=€  233.11
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  13937.79 , over-consume=kWh  32752.7 , under-consume=kWh  18814.91 , Total_cost=€  635.96 , overcost=€  1162.1 , Charging_events=  783 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  83.41
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  12996.65 , over-consume=kWh  31760.33 , under-consume=kWh  18763.68 , Total_cost=€  614